# Simple QUBO Example
### James Saslow
### 12/5/2023

In this notebook, we will add the 'quadraticness' back into our cost function to solve a simple QUBO on DWave hybrid solvers!

As a recap, the goal to solving QUBO problems is to find the optimal solutions to the following cost function:


### $C(X) = \sum_{i} W_{i} x_{i} + \sum_{\{i,j\}\epsilon \mathbb{S}} w_{i,j} x_{i} x_{j}$ 

A particular cost function we will focus on is


### $C(|x_{0}x_{1}>) = 2 x_{0} - x_{1}  -3 x_{0} x_{1} $

Notice how $2 x_{0} - x_{1}$ is the linear part and $-3 x_{0} x_{1}$ is the quadratic part of the QUBO

We want to find
- 1) The binary bit string $|x_{0}x_{1}>$ that minimizes the cost function $C(X)$
- 2) The binary bit string $|x_{0}x_{1}>$ that maximizes the cost function $C(X)$


Let's brute-force compute all outcomes ourselves so we know what to expect

#### $C(|00>) = 2*0 -0 - 3*0*0 = 0$
#### $C(|01>) = 2*0 - 1 - 3*0*1 = -1$
#### $C(|10>) = 2 - 0 - 3* 1 * 0 = 2$
#### $C(|11>) = 2 - 1 - 3*1*1 = -2$

Thus, we see by a classic brute-force search that $|11>$ minimizes the cost function and $|10>$ maximizes the cost function. Let's cross-check our results with DWave!

### Task 1) Finding the binary bit string that minimizes $C(X)$ using DWave

In [8]:
# Importing DWave Packages
from dwave.system import DWaveSampler, EmbeddingComposite
from dimod import BinaryQuadraticModel

In [2]:
# Defining the QUBO parameters

items   = [0,1]  # Binary Variable Labeling
weights = [2,-1] # W weights associated with each binary variable

x = ['Item_0', 'Item_1'] # More Binary Variable Labeling

In [4]:
# Encoding the QUBO in DWave solvers

bqm = BinaryQuadraticModel("BINARY") # Encoding QUBO

# Encoding the Linear Terms
for i in items:
    bqm.add_variable(x[i], weights[i])

# Encoding the Quadratic Terms
bqm.add_interaction('Item_0','Item_1', -3) 
    
bqm

BinaryQuadraticModel({'Item_0': 2.0, 'Item_1': -1.0}, {('Item_1', 'Item_0'): -3.0}, 0.0, 'BINARY')

In [5]:
# Solve the problem using D-Wave
sampler = EmbeddingComposite(DWaveSampler())
sampleset = sampler.sample(bqm, num_reads=10)

In [6]:
# Post-Processing Solver Output
opt_solution = sampleset.first.sample # Binary Bit-String that minimizes the cost function
opt_value = sampleset.first.energy    # Minimized Cost function value

In [7]:
print(opt_solution)
print('C_min = ', opt_value)

{'Item_0': 1, 'Item_1': 1}
C_min =  -2.0


### DWave solvers suggests that the binary bit string $|11>$ minimizes the cost function $C(X)$ with $C(|11>) = -2$

### Task 1) Finding the binary bit string that maximizes $C(X)$ using DWave

We must multiply our entire cost function by $-1$ to flip minimums to maximum and maximums to minimums in order to form $C'(X)$. The minimum of $C'(X)$ is the maximum of $C(X)$.

In [14]:
# Defining the QUBO parameters

items   = [0,1]  # Binary Variable Labeling

# Flipping the signs on each element in 'weights' to create C(X)
weights = [-2,1] # W weights associated with each binary variable

x = ['Item_0', 'Item_1'] # More Binary Variable Labeling

In [15]:
# Encoding the QUBO in DWave solvers

bqm = BinaryQuadraticModel("BINARY") # Encoding QUBO

# Encoding the Linear Terms
for i in items:
    bqm.add_variable(x[i], weights[i])

# Encoding the Quadratic Terms
bqm.add_interaction('Item_0','Item_1', 3) 
    
bqm

BinaryQuadraticModel({'Item_0': -2.0, 'Item_1': 1.0}, {('Item_1', 'Item_0'): 3.0}, 0.0, 'BINARY')

In [16]:
# Solve the problem using D-Wave
sampler = EmbeddingComposite(DWaveSampler())
sampleset = sampler.sample(bqm, num_reads=10)

In [17]:
# Post-Processing Solver Output
opt_solution = sampleset.first.sample # Binary Bit-String that minimizes the cost function
opt_value = sampleset.first.energy    # Minimized Cost function value

In [18]:
print(opt_solution)
print('C_min = ', opt_value)

{'Item_0': 1, 'Item_1': 0}
C_min =  -2.0


### DWave solvers suggests that the binary bit string $|10>$ maximizes the cost function $C(X)$ with $C(|10>) = 2$

------------------------------------------------------

To summarize our findings, 

we found that both classically and with DWave solvers that the solutions to the following QUBO are

### $C(|x_{0}x_{1}>) = 2 x_{0} - x_{1}  -3 x_{0} x_{1} $

### Min Solution: $X_{min} = |11>$, $C(X_{min}) = -2$

### Max Solution: $X_{max} = |10>$, $C(X_{max}) = 2$

Now, we are well-equiped so solve NP-Hard problems that have quadratic terms in their cost function -- like the *Max-cut problem*!

More soon -- Stay tuned!

### Sources

[1] “Binary quadratic models#,” Binary Quadratic Models - Ocean Documentation 6.7.1 documentation, https://docs.ocean.dwavesys.com/en/stable/concepts/bqm.html (accessed Dec. 11, 2023). 

[2] “Quantum programming tutorial | D-wave qubits 2021,” YouTube, https://www.youtube.com/watch?v=jTDnGox0c9Y (accessed Dec. 11, 2023). 